In [7]:
!pip install -q transformers pandas torch


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [8]:
import pandas as pd
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
from tqdm import tqdm
import torch


In [9]:
model_path = "gopi30/english-to-tamil-stage1"

model = M2M100ForConditionalGeneration.from_pretrained(model_path).to("cuda")  # use "cpu" if no GPU
tokenizer = M2M100Tokenizer.from_pretrained(model_path)


In [10]:
def translate_en_to_ta(text):
    tokenizer.src_lang = "en"
    encoded = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to("cuda")
    generated = model.generate(**encoded, forced_bos_token_id=tokenizer.get_lang_id("ta"), max_length=512)
    return tokenizer.decode(generated[0], skip_special_tokens=True)


In [11]:
def batch_translate(text_list):
    translated_texts = []
    for text in tqdm(text_list):
        if isinstance(text, str) and text.strip():
            translated_texts.append(translate_en_to_ta(text.strip()))
        else:
            translated_texts.append("")
    return translated_texts


In [12]:
df = pd.read_csv("medquad.csv")  # replace with your path if different
df['question'] = df['question'].fillna("")
df['answer'] = df['answer'].fillna("")
df.head()


,question,answer,source,focus_area
0,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...,NIHSeniorHealth,Glaucoma
1,What causes Glaucoma ?,"Nearly 2.7 million people have glaucoma, a lea...",NIHSeniorHealth,Glaucoma
2,What are the symptoms of Glaucoma ?,Symptoms of Glaucoma Glaucoma can develop in ...,NIHSeniorHealth,Glaucoma
3,What are the treatments for Glaucoma ?,"Although open-angle glaucoma cannot be cured, ...",NIHSeniorHealth,Glaucoma
4,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...,NIHSeniorHealth,Glaucoma


In [13]:
questions_tamil = batch_translate(df['question'].tolist())


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16412/16412 [1:10:36<00:00,  3.87it/s]


In [15]:
answers_tamil = batch_translate(df['answer'].tolist())

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16412/16412 [5:37:02<00:00,  1.23s/it]


In [16]:
translated_df = pd.DataFrame({
    'question_tamil': questions_tamil,
    'answer_tamil': answers_tamil
})

translated_df.head()


,question_tamil,answer_tamil
0,(அது) க்ளவுகோமா என்ன?,ஆனால் க்ளூகோமியா என்பது கண் நரம்பின் பார்வைக்க...
1,குளோகோமைக்கு என்ன காரணம்?,"இதில், 40 வயதுக்கும் மேற்பட்ட ஆபிரிக்க அமெரிக்..."
2,க்ளவுகோமாவின் அறிகுறிகள் என்ன?,"சிகிச்சை இல்லாமல், சிகிச்சை இல்லாமல், சிகிச்சை..."
3,க்ளவுகோமாவுக்கு என்ன சிகிச்சைகள்?,க்ளாகோமியாவைப் பற்றிய ஆய்வுகள் மற்றும் அறுவைச்...
4,(அது) க்ளவுகோமா என்ன?,க்ளவுகோமம் என்பது கண் நரம்பின் ஆப்டிக் நரம்பை ...


In [17]:
translated_df.to_csv("medquad_tamil.csv", index=False)
